In [1]:
import networkx as nx
import osmnx as ox
import geopandas as gpd
%matplotlib inline
ox.config(log_console=True, use_cache=True)
ox.__version__

'0.12.1'

In [2]:
from cartoframes.viz import (Map, Layer, basemaps, popup_element, basic_style, 
                             color_category_style, color_bins_style, default_legend, color_continuous_style,
                             color_bins_legend, category_widget, 
                             formula_widget,  histogram_widget)

In [3]:
dc_boundary = gpd.read_file("Washington_DC_Boundary-shp")

In [8]:
dc_boundary.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
dc_boundary_geom = dc_boundary.unary_union

In [13]:
bus_route_extent = gpd.read_file("gis_bsrt_lin_extent")

In [14]:
bus_route_extent.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [15]:
extent_geom = bus_route_extent.geometry[0]

In [9]:
#use dc_boundary or bus_line_extent
G = ox.graph_from_polygon(dc_boundary_geom,network_type='drive')

NumExpr defaulting to 3 threads.


In [23]:
# filepath = '/home/work/data/dc_boundary_network.graphml'
# ox.save_graphml(G, filepath)
# G = ox.load_graphml(filepath)

In [10]:
edges = ox.graph_to_gdfs(G, nodes=False)

In [17]:
nodes, edges = ox.graph_to_gdfs(G, nodes=True)

In [18]:
nodes

,y,x,osmid,highway,ref,geometry
49741825,38.926705,-76.971901,49741825,NaN,NaN,POINT (-76.97190 38.92670)
49774596,38.941291,-77.070760,49774596,NaN,NaN,POINT (-77.07076 38.94129)
49741829,38.900090,-76.953055,49741829,NaN,NaN,POINT (-76.95306 38.90009)
49774604,38.941226,-77.072418,49774604,NaN,NaN,POINT (-77.07242 38.94123)
49741856,38.898653,-76.951200,49741856,NaN,NaN,POINT (-76.95120 38.89865)
...,...,...,...,...,...,...
647069687,38.929148,-77.017945,647069687,NaN,NaN,POINT (-77.01795 38.92915)
49741817,38.942939,-77.041272,49741817,NaN,NaN,POINT (-77.04127 38.94294)
49741820,38.942968,-77.043082,49741820,NaN,NaN,POINT (-77.04308 38.94297)
4303519741,38.890416,-77.031962,4303519741,traffic_signals,NaN,POINT (-77.03196 38.89042)


In [20]:
Map(Layer(nodes.iloc[:1000],
         popup_click=popup_element('osmid')))

In [57]:
edges.loc[:,'osmid_str'] = (edges.loc[:,'osmid']
                            .apply(lambda x: 
                                   ", ".join([str(item) for item in x]) 
                                   if type(x) == list 
                                   else str(x)))
edges.loc[:,'name_str'] = (edges.loc[:,'name']
                            .apply(lambda x: ", ".join(x) 
                                   if type(x) == list 
                                   else str(x)))

edges.loc[:,'highway_str'] = (edges.loc[:,'highway']
                            .apply(lambda x: ", ".join(x) 
                                   if type(x) == list 
                                   else str(x)))

edges.loc[:,'ref_str'] = (edges.loc[:,'ref']
                            .apply(lambda x: ", ".join(x) 
                                   if type(x) == list 
                                   else str(x)))

In [58]:
edges.columns

Index(['u', 'v', 'key', 'osmid', 'highway', 'oneway', 'length', 'geometry',
       'name', 'ref', 'bridge', 'maxspeed', 'lanes', 'access', 'junction',
       'tunnel', 'service', 'width', 'osmid_str', 'name_str', 'highway_str',
       'ref_str'],
      dtype='object')

In [59]:
edges_for_json = edges[['u', 'v', 'key','length', 'geometry','osmid_str', 'name_str', 'highway_str',
       'ref_str']]

In [60]:
edges_for_json.to_file("osm_network_bus_extent.geojson", driver="GeoJSON")

osm_network_bus_extent.geojson: No such file or directory
driver GeoJSON does not support creation option ENCODING
